# But Why...?

Because it feels ridiculous! Also, if you can find complex patterns in multidimensional data can you find relatively simple patterns from a single input?

# Generating the Data

In [ ]:
import numpy as np

def fizz_buzz_encoder(n):
    '''
    Encode any given number to the FizzBuzz representation. That is 'fizz' if
    divisible by 3, 'buzz' if divisible by 5, 'fizzbuzz' if divisible by both
    3 and 5, or simply the number given. The encoded output is a vector 
    representing the output; [1,0] meaning a "fizz", [0,1] a "buzz", and [1,1]
    a "fizzbuzz". Note that the zero vector represents no word (just the 
    number).
    '''
    # Default to nothing in array
    vector = np.array([0,0])
    if n % 3 == 0: 
        vector += np.array([0,1])
    if n % 5 == 0:
        vector += np.array([1,0])
        
    return vector

def fizz_buzz_decoder(n_vector, n_default=''):
    '''
    Decodes the fizzbuzz vector representation to an output. See 
    fizz_buzz_encoder for details on the representation. n_default is the 
    default string representation if 'fizz', 'buzz' or 'fizzbuzz' should not
    be used in the string representation (usually just the number)
    '''
    # Use the vector to create the different 
    output = n_vector[0] * 'fizz' + n_vector[1] * 'buzz'
    # If zero vector, it should printout the default number
    if n_vector.sum() == 0:
        output = str(n_default)
    return output